In [ ]:
! pip install kaggle-environments --upgrade -q

In [ ]:
%%writefile decaying_e_greedy.py

import numpy as np

arm_selections = None
arm_mean_rewards = None
epsilon = 1
arm = None
total_rewards = 0

def agent(observation, configuration):
    global arm_selections, arm_mean_rewards, epsilon, arm, total_rewards
    
    n_bandits = configuration.banditCount
    
    if observation.step == 0:
        arm_selections = np.zeros(n_bandits)
        arm_mean_rewards = np.zeros(n_bandits)
    
    if observation.step > 0:
        epsilon *= 0.99
        reward = observation.reward - total_rewards
        total_rewards = observation.reward
        
        current_arm_reward = arm_mean_rewards[arm] * arm_selections[arm]
        arm_selections[arm] += 1
        new_mean = (current_arm_reward + reward) / arm_selections[arm]
        arm_mean_rewards[arm] = new_mean
    
    if np.random.binomial(1, epsilon, 1):
        arm = np.random.randint(0, n_bandits)
    else:
        arm = np.argmax(arm_mean_rewards)
    
    return int(arm)

In [ ]:
from kaggle_environments import make
env = make("mab", debug=True)

env.reset()
env.run(["../input/santa-2020/submission.py", "decaying_e_greedy.py"])
env.render(mode="ipython", width=800, height=500)